# Libraries

In [6]:
import pandas as pd
import numpy as np
import requests

import warnings
warnings.filterwarnings("ignore")

from pyjstat import pyjstat
from ecbdata import ecbdata

import matplotlib.pyplot as plt

from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.worksheet.hyperlink import Hyperlink
from datetime import date

from calendar import monthrange

%matplotlib inline

# Functions

In [7]:
def to_last_day_of_month(date_str):
    date = pd.to_datetime(date_str, format='%Y-%m')
    last_day = monthrange(date.year, date.month)[1]
    return date.replace(day=last_day)

In [55]:
def extract_data_from_bank_pt(series_id, variable_name):
    """ 
    Function to extract data from BPSTAT API.

    Arguments: series_id int
             variable_name str.
             If variable_name is None, variable_name is set to urls label.

    Returns:   pandas dataframe with Date and variable_name columns
    """
    
    BPSTAT_API_URL="https://bpstat.bportugal.pt/data/v1"

    url = f"{BPSTAT_API_URL}/series/?lang=EN&series_ids={series_id}"
    series_info = requests.get(url).json()[0]

    domain_id = series_info["domain_ids"][0]
    dataset_id = series_info["dataset_id"]

    dataset_url = f"{BPSTAT_API_URL}/domains/{domain_id}/datasets/{dataset_id}/?lang=EN&series_ids={series_id}"
    dataset = pyjstat.Dataset.read(dataset_url)
    df = dataset.write('dataframe')

    df['Date'] = pd.to_datetime(df['Date'])
    if variable_name is None:
        variable_name = series_info['label']

    df = df.rename(columns={'value': variable_name})
    df = df[['Date', variable_name]]

    return df


def preprocess_data_from_bpt(series_id):
    df = extract_data_from_bank_pt(series_id, 'None').sort_values(by='Date', ascending=False)
    df['is_month_end'] = df['Date'].dt.is_month_end
    df = df[df['is_month_end']]
    df = df.drop(columns=['is_month_end'])
    df['Date'] = df['Date'].dt.date
    
    return df

In [9]:
def extract_data_from_ecb(key, start_date='2020-01', novo_nome_coluna=None):
    """ 
    Function to extract data from ECB.

    Arguments: key str: URL key
               start_date str:  start date

    Returns:   pandas dataframe with TIME_PERIOD and OBS_VALUE columns
    """
    
    df = ecbdata.get_series(key,
                        start=start_date, detail='dataonly')
    
    df.TIME_PERIOD = df['TIME_PERIOD'].apply(to_last_day_of_month)
    df['TIME_PERIOD'] = df['TIME_PERIOD'].dt.strftime('%d/%m/%Y')

    df = df[['TIME_PERIOD', 'OBS_VALUE']]
    
    if novo_nome_coluna:
        df = df.rename(columns={'OBS_VALUE': novo_nome_coluna})
        
    df = df.rename(columns={'TIME_PERIOD': 'DATE'})
        
    return df

# GET and PREPROCESS DATA

In [57]:
# extracting data with original column names
df2 = extract_data_from_ecb('FM.M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA', start_date='2023-05-01', novo_nome_coluna= 'Euribor 3-month - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA)')
df3 = extract_data_from_ecb('FM.M.U2.EUR.RT.MM.EURIBOR6MD_.HSTA', start_date='2023-05-01', novo_nome_coluna= 'Euribor 6-month - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR6MD_.HSTA)')
df4 = extract_data_from_ecb('FM.M.U2.EUR.RT.MM.EURIBOR1YD_.HSTA', start_date='2023-05-01', novo_nome_coluna= 'Euribor 12-month - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR1YD_.HSTA)')

In [58]:
df5 = preprocess_data_from_bpt(5721524)
df6 = preprocess_data_from_bpt(5739035)
df7 = preprocess_data_from_bpt(5739374)
df8 = preprocess_data_from_bpt(12518314)
df9 = preprocess_data_from_bpt(12457932)
df10 = preprocess_data_from_bpt(12457933)
df11 = preprocess_data_from_bpt(12587302)
df12 = preprocess_data_from_bpt(12533712)
df13 = preprocess_data_from_bpt(12457924)
df14 = preprocess_data_from_bpt(12457925)
df15 = preprocess_data_from_bpt(12562641)
df16 = preprocess_data_from_bpt(12533735)

In [105]:
def preprocess_euribor_data(series_key, novo_nome_coluna):
    df_aux = extract_data_from_ecb(series_key, start_date='2009-01-01', novo_nome_coluna=novo_nome_coluna)
    
    df_aux['DATE'] = pd.to_datetime(df_aux['DATE'])
    # Extrair o mês da coluna 'date'
    df_aux['month'] = df_aux['DATE'].dt.month

    # Filtrar apenas as linhas onde o mês é março (3), junho (6), setembro (9) ou dezembro (12)
    df_aux= df_aux[df_aux['month'].isin([3, 6, 9, 12])]

    # Remover a coluna auxiliar, se não for mais necessária
    df_aux = df_aux.drop(columns=['month'])
    df_aux = df_aux.rename(columns={'DATE': 'Date'})
    df_aux['Date'] = df_aux['Date'].dt.date
    
    return df_aux

In [182]:
df_aux = preprocess_euribor_data('FM.M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA', 'EUR 3M')
df_aux = df_aux.merge(preprocess_euribor_data('FM.M.U2.EUR.RT.MM.EURIBOR6MD_.HSTA', 'EUR 6M'), on='Date', how='left')
df_aux = df_aux.merge(preprocess_euribor_data('FM.M.U2.EUR.RT.MM.EURIBOR1YD_.HSTA', 'EUR 12M'), on='Date', how='left')

df_aux = df_aux.merge(df5, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'IPC (tvh)'})
df_aux = df_aux.merge(df6, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'IPH (tvh)'})
df_aux = df_aux.merge(df7, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'Tx. Desemp.'})
df_aux = df_aux.merge(df8, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'PIB (tvh)'})
df_aux = df_aux.merge(df9, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'Endividamento do setor não financeiro (M€)'})
df_aux = df_aux.merge(df10, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'Endividamento do setor não financeiro (%PIB)'})
df_aux = df_aux.merge(df11, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'Custo dos financiamentos obtidos - Empresas'})
df_aux = df_aux.merge(df12, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'TAA de novos empréstimos ao setor não financeiro'})
df_aux = df_aux.merge(df13, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'Endividamento dos particulares (M€)'})
df_aux = df_aux.merge(df14, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'Endividamento dos particulares (%PIB)'})
df_aux = df_aux.merge(df15, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'Taxa de poupança dos particulares (%RD)'})
df_aux = df_aux.merge(df16, on='Date', how='left')
df_aux = df_aux.rename(columns={'None': 'TAA de novos empréstimos à habitação',
                               'Date': 'Data'})

In [183]:
def add_percentage_symbol(x):
    if isinstance(x, (int, float)):
        return f"{x:.2f}%"

for col in df_aux1.columns:
    if col not in ['Data', 'Endividamento do setor não financeiro (M€)', 'Endividamento dos particulares (M€)']:
        df_aux[col] = df_aux[col].apply(add_percentage_symbol)

In [185]:
df_aux.replace('nan%', '', inplace=True)

In [187]:
df_aux = df_aux.fillna('')

In [211]:
#df_aux

# Write excel

In [213]:
file_name = 'Dados Macroeconómicos vRicardo_.xlsx'

save_path = "C:/Users/B28658/Desktop/" + file_name

In [214]:
sheet_name_trim = 'Indicadores (trim)'
sheet_name1 = 'Update'
sheet_name2 = 'EUR 3M'
sheet_name3 = 'EUR 6M'
sheet_name4 = 'EUR 12M'
sheet_name5 = 'IPC (tvh)'
sheet_name6 = 'IPH (tvh)'
sheet_name7 = 'Tx. Desemp.'
sheet_name8 = 'PIB (tvh)'
sheet_name9 = 'Endiv. Empresas'
sheet_name10 = 'Endiv. Empresas %PIB'
sheet_name11 = 'Custos Financ. Empresas'
sheet_name12 = 'TAA Empresas'
sheet_name13 = 'Endiv. Particulares'
sheet_name14 = 'Endiv. Particulares %PIB'
sheet_name15 = 'Tx. Poup. Particulares'
sheet_name16 = 'TAA Particulares'



def ajustar_altura_primeira_linha(worksheet, altura):
    worksheet.row_dimensions[1].height = altura

def ajustar_largura_colunas(worksheet, largura):
    for col in worksheet.columns:
        col_letter = col[0].column_letter
        worksheet.column_dimensions[col_letter].width = largura
        
def aplicar_wrap_text_primeira_linha(worksheet):
    for cell in worksheet[1]:
        cell.alignment = Alignment(wrap_text=True)


def formatar_sheet(worksheet, sheet_name):
    header_font = Font(bold=True, name='Arial', size=11)
    white_fill = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")
    hyperlink_font = Font(color="07cac5", name='Arial', underline="single")
    arial_font = Font(name='Arial', size=11)
    
    max_row = 60 
    max_col = 24 

    # Preenchendo todas as células com fundo branco e definindo a fonte Arial
    for row in worksheet.iter_rows(min_row=1, max_row=max_row+50, min_col=1, max_col=max_col+30):
        for cell in row:
            cell.fill = white_fill
            cell.font = arial_font

    if sheet_name == 'Update':
        
        worksheet.sheet_properties.tabColor = "dcf7f3"
        
        # Adicionar textos específicos
        worksheet['A1'] = "FONTES:                                   UPDATED:  " + str(date.today())  
        worksheet['A1'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A3'] = "• Euribor 3M, 6M, 12M"
        worksheet['A3'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A4'] = "Fonte: ECB Data Portal"
        worksheet['A4'].font = Font(name='Arial', size=12)
        
        worksheet['A5'] = "FM.M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA | ECB Data Portal (europa.eu)"
        worksheet['A5'].hyperlink = Hyperlink(ref='A5', target='https://data.ecb.europa.eu/data/datasets/FM/FM.M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA')
        worksheet['A5'].font = hyperlink_font
        
        worksheet['A6'] = "FM.M.U2.EUR.RT.MM.EURIBOR6MD_.HSTA | ECB Data Portal (europa.eu)"
        worksheet['A6'].hyperlink = Hyperlink(ref='A6', target='https://data.ecb.europa.eu/data/datasets/FM/FM.M.U2.EUR.RT.MM.EURIBOR6MD_.HSTA')
        worksheet['A6'].font = hyperlink_font
        
        worksheet['A7'] = "FM.M.U2.EUR.RT.MM.EURIBOR1YD_.HSTA | ECB Data Portal (europa.eu)"
        worksheet['A7'].hyperlink = Hyperlink(ref='A7', target='https://data.ecb.europa.eu/data/datasets/FM/FM.M.U2.EUR.RT.MM.EURIBOR1YD_.HSTA')
        worksheet['A7'].font = hyperlink_font
        
        worksheet['A9'] = "• Indíce de Preços no Consumidor (Taxa de Variação Homóloga)"
        worksheet['A9'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A10'] = "Fonte: BPStat"
        worksheet['A10'].font = Font(name='Arial', size=12)
        worksheet['A11'] = "IPC total-taxa de variação homóloga | BPstat (bportugal.pt)"
        worksheet['A11'].hyperlink = Hyperlink(ref='A11', target='https://bpstat.bportugal.pt/serie/5721524')
        worksheet['A11'].font = hyperlink_font
        
        worksheet['A13'] = "• Índice de Preços de Habitação (Taxa de Variação Homóloga)"
        worksheet['A13'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A14'] = "Fonte: BPStat"
        worksheet['A14'].font = Font(name='Arial', size=12)
        worksheet['A15'] = "Índice de preços da habitação-Total-TVH-Trimestral | BPstat (bportugal.pt)"
        worksheet['A15'].hyperlink = Hyperlink(ref='A15', target='https://bpstat.bportugal.pt/serie/5739035')
        worksheet['A15'].font = hyperlink_font
        
        worksheet['A17'] = "• Taxa de Desemprego"
        worksheet['A17'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A18'] = "Fonte: BPStat"
        worksheet['A18'].font = Font(name='Arial', size=12)
        worksheet['A19'] = "Taxa de desemprego 16 a 74 anos-Total-Trim | BPstat (bportugal.pt)"
        worksheet['A19'].hyperlink = Hyperlink(ref='A19', target='https://bpstat.bportugal.pt/serie/5739374')
        worksheet['A19'].font = hyperlink_font
                
        worksheet['A21'] = "• PIB (Taxa de Variação Homóloga)"
        worksheet['A21'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A22'] = "Fonte: BPStat"
        worksheet['A22'].font = Font(name='Arial', size=12)
        worksheet['A23'] = "PIB a preços de mercado-Trim-Dados encadeados volume-TVH (vcsc) | BPstat (bportugal.pt)"
        worksheet['A23'].hyperlink = Hyperlink(ref='A23', target='https://bpstat.bportugal.pt/serie/12518314')
        worksheet['A23'].font = hyperlink_font
        
        worksheet['A25'] = "• Endividamento do setor não financeiro"
        worksheet['A25'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A26'] = "Fonte: BPStat"
        worksheet['A26'].font = Font(name='Arial', size=12)
        worksheet['A27'] = "Endividamento do setor não financeiro | BPstat (bportugal.pt)"
        worksheet['A27'].hyperlink = Hyperlink(ref='A27', target='https://bpstat.bportugal.pt/serie/12457932')
        worksheet['A27'].font = hyperlink_font
        
        worksheet['A29'] = "• Endividamento do setor não financeiro em %PIB"
        worksheet['A29'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A30'] = "Fonte: BPStat"
        worksheet['A30'].font = Font(name='Arial', size=12)
        worksheet['A31'] = "Endividamento do setor não financeiro em %PIB | BPstat (bportugal.pt)"
        worksheet['A31'].hyperlink = Hyperlink(ref='A31', target='https://bpstat.bportugal.pt/serie/12457933')
        worksheet['A31'].font = hyperlink_font
        
        worksheet['A33'] = "• Custo dos financiamentos obtidos - Empresas - Trimestral - %"
        worksheet['A33'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A34'] = "Fonte: BPStat"
        worksheet['A34'].font = Font(name='Arial', size=12)
        worksheet['A35'] = "Custo dos financiamentos obtidos - Empresas - Trimestral - % | BPstat (bportugal.pt)"
        worksheet['A35'].hyperlink = Hyperlink(ref='A35', target='https://bpstat.bportugal.pt/serie/12587302')
        worksheet['A35'].font = hyperlink_font
        
        worksheet['A37'] = "• Taxa de juro (TAA) de novos empréstimos ao setor não financeiro (exceto administrações públicas)"
        worksheet['A37'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A38'] = "Fonte: BPStat"
        worksheet['A38'].font = Font(name='Arial', size=12)
        worksheet['A39'] = "Taxa de juro (TAA) de novos empréstimos ao setor não financeiro (exceto administrações públicas) | BPstat (bportugal.pt)"
        worksheet['A39'].hyperlink = Hyperlink(ref='A39', target='https://bpstat.bportugal.pt/serie/12533712')
        worksheet['A39'].font = hyperlink_font
        
        worksheet['A41'] = "• Endividamento dos particulares"
        worksheet['A41'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A42'] = "Fonte: BPStat"
        worksheet['A42'].font = Font(name='Arial', size=12)
        worksheet['A43'] = "Endividamento dos particulares | BPstat (bportugal.pt)"
        worksheet['A43'].hyperlink = Hyperlink(ref='A43', target='https://bpstat.bportugal.pt/serie/12457924')
        worksheet['A43'].font = hyperlink_font
        
        worksheet['A45'] = "• Endividamento dos particulares em %PIB"
        worksheet['A45'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A46'] = "Fonte: BPStat"
        worksheet['A46'].font = Font(name='Arial', size=12)
        worksheet['A47'] = "Endividamento dos particulares em %PIB | BPstat (bportugal.pt)"
        worksheet['A47'].hyperlink = Hyperlink(ref='A47', target='https://bpstat.bportugal.pt/serie/12457925')
        worksheet['A47'].font = hyperlink_font
        
        worksheet['A49'] = "• SEP-Taxa de poupança dos particulares-Trim-%RD"
        worksheet['A49'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A50'] = "Fonte: BPStat"
        worksheet['A50'].font = Font(name='Arial', size=12)
        worksheet['A51'] = "SEP-Taxa de poupança dos particulares-Trim-%RD | BPstat (bportugal.pt)"
        worksheet['A51'].hyperlink = Hyperlink(ref='A51', target='https://bpstat.bportugal.pt/serie/12562641')
        worksheet['A51'].font = hyperlink_font
        
        worksheet['A53'] = "• Taxa de juro (TAA) de novos empréstimos à habitação"
        worksheet['A53'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A54'] = "Fonte: BPStat"
        worksheet['A54'].font = Font(name='Arial', size=12)
        worksheet['A55'] = "Taxa de juro (TAA) de novos empréstimos à habitação | BPstat (bportugal.pt)"
        worksheet['A55'].hyperlink = Hyperlink(ref='A55', target='https://bpstat.bportugal.pt/serie/12533735')
        worksheet['A55'].font = hyperlink_font
        
        worksheet['A57'] = "Calendário de publicações BPStat"
        worksheet['A57'].font = Font(bold=True, name='Arial', size=12)
        worksheet['A58'] = "Calendário BPstat (bportugal.pt)"
        worksheet['A58'].hyperlink = Hyperlink(ref='A58', target='https://bpstat.bportugal.pt/api/media/files/Calendario_BPstat_data.html')
        worksheet['A58'].font = hyperlink_font

        worksheet.column_dimensions['A'].width = 100

        # Alinhar valores da coluna A à esquerda
        for row in worksheet.iter_rows(min_row=1, max_row=max_row, min_col=1, max_col=1):
            for cell in row:
                cell.alignment = Alignment(horizontal='left', vertical='center')
    elif sheet_name in [sheet_name2, sheet_name3, sheet_name4]:
        
        worksheet.sheet_properties.tabColor = "dcf7f3"
        # Aplicar formatação aos cabeçalhos
        for cell in worksheet[1]:
            cell.font = header_font
            cell.alignment = Alignment(horizontal='center', vertical='center')

        # Centralizar valores das colunas
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=worksheet.max_column):
            for cell in row:
                if cell.column_letter == 'B':
                    cell.alignment = Alignment(horizontal='left', vertical='center')
                else:
                    cell.alignment = Alignment(horizontal='center', vertical='center')

        # Ajustar a largura das colunas
        worksheet.column_dimensions['A'].width = 15  
        worksheet.column_dimensions['B'].width = 123
    
    elif sheet_name == 'IPC (tvh)':
        
        worksheet.sheet_properties.tabColor = "dcf7f3"
        
        worksheet['B1'] = "5721524"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/5721524')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')
        
        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')
        
        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')
        
        worksheet['B2'] = "IPC total-taxa de variação homóloga"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')
        
        worksheet['B3'] = "Taxa de variação homóloga"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')
        
        worksheet['B4'] = "Percentagem"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')
        
        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 50

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'IPH (tvh)':
        
        worksheet.sheet_properties.tabColor = "dcf7f3"
        
        worksheet['B1'] = "5739035"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/5739035')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "Índice de preços da habitação-Total-TVH-Trimestral"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "Taxa de variação homóloga"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Percentagem"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 50

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'Tx. Desemp.':
        
        worksheet.sheet_properties.tabColor = "dcf7f3"
        
        worksheet['B1'] = "5739374"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/5739374')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "Taxa de desemprego 16 a 74 anos-Total-Trim"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "Rácio"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Percentagem"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 50

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'PIB (tvh)':
        
        worksheet.sheet_properties.tabColor = "dcf7f3"
        
        worksheet['B1'] = "12518314"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/12518314')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "PIB a preços de mercado-Trim-Dados encadeados volume-TVH (vcsc)"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "Taxa de variação homóloga"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Percentagem"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 70

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'Endiv. Empresas':
        worksheet.sheet_properties.tabColor = 'fad9e3'
        
        worksheet['B1'] = "12457932"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/12457932')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "Endividamento do setor não financeiro"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "Valor"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Milhões de euros"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 50

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'Endiv. Empresas %PIB':
        
        worksheet.sheet_properties.tabColor = 'fad9e3'
        
        worksheet['B1'] = "12457933"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/12457933')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "Endividamento do setor não financeiro em %PIB"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "% do PIB"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Percentagem"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 50

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'Custos Financ. Empresas':
        
        worksheet.sheet_properties.tabColor = 'fad9e3'
        
        worksheet['B1'] = "12587302"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/12587302')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "Custo dos financiamentos obtidos - Empresas - Trimestral - %"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "Valor"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Percentagem"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 60

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'TAA Empresas':
        
        worksheet.sheet_properties.tabColor = 'fad9e3' #faf5d9
        
        worksheet['B1'] = "12533712"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/12533712')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "Taxa de juro (TAA) de novos empréstimos ao setor não financeiro (exceto administrações públicas)"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "Taxa acordada anualizada"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Percentagem"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 100

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'Endiv. Particulares':
        
        worksheet.sheet_properties.tabColor = 'faf5d9'
        
        worksheet['B1'] = "12457924"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/12457924')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "Endividamento dos particulares"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "Valor"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Milhões de euros"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 50

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'Endiv. Particulares %PIB':
        
        worksheet.sheet_properties.tabColor = 'faf5d9'
        worksheet['B1'] = "12457925"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/12457925')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "Endividamento dos particulares em %PIB"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "% do PIB"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Percentagem"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 50

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'Tx. Poup. Particulares':
        
        worksheet.sheet_properties.tabColor = 'faf5d9'
        worksheet['B1'] = "12562641"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/12562641')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "SEP-Taxa de poupança dos particulares-Trim-%RD"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "% do rendimento disponível"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Percentagem"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 50

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'TAA Particulares':
        
        worksheet.sheet_properties.tabColor = 'faf5d9'
        worksheet['B1'] = "12533735"
        worksheet['B1'].hyperlink = Hyperlink(ref='B1', target='https://bpstat.bportugal.pt/serie/12533735')
        worksheet['B1'].font = Font(color="07cac5", name='Arial', underline="single", bold=True)
        worksheet['B1'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A3'] = "Métrica"
        worksheet['A3'].font = header_font
        worksheet['A3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['A4'] = "Unidade de Medida"
        worksheet['A4'].font = header_font
        worksheet['A4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B2'] = "Taxa de juro (TAA) de novos empréstimos à habitação"
        worksheet['B2'].font = header_font
        worksheet['B2'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B3'] = "Taxa acordada anualizada"
        worksheet['B3'].font = header_font
        worksheet['B3'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet['B4'] = "Percentagem"
        worksheet['B4'].font = header_font
        worksheet['B4'].alignment = Alignment(horizontal='center', vertical='center')

        worksheet.column_dimensions['A'].width = 25  
        worksheet.column_dimensions['B'].width = 50

        # Centralizar os valores nas colunas A e B
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')
                
    elif sheet_name == 'Indicadores (trim)':
        # Formatar a primeira linha
        for cell in worksheet[1]:
            cell.font = header_font
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

        # Definir a coluna A como negrito
        for cell in worksheet['A']:
            cell.font = Font(bold=True, name='Arial', size=11)

        ajustar_largura_colunas(worksheet, 16)  # Ajustar a largura de todas as colunas para 20
        ajustar_altura_primeira_linha(worksheet, 60)  # Ajustar a altura da primeira linha para 60

        # Aplicar wrap text na primeira linha
        #aplicar_wrap_text_primeira_linha(worksheet)

        # Centralizar valores das colunas a partir da segunda linha
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=worksheet.max_column):
            for cell in row:
                cell.alignment = Alignment(horizontal='center', vertical='center')



# Usando o ExcelWriter para escrever múltiplos DataFrames em um único arquivo Excel
with pd.ExcelWriter(save_path, engine='openpyxl') as writer:
    # Criar a primeira sheet com texto
    writer.book.create_sheet(sheet_name_trim)
    writer.book.create_sheet(sheet_name1)
    writer.sheets[sheet_name1] = writer.book[sheet_name1]
    formatar_sheet(writer.sheets[sheet_name1], sheet_name1)

    # Escrever DataFrames nas outras sheets
    df_aux.to_excel(writer, sheet_name=sheet_name_trim, index=False)
    df2.to_excel(writer, sheet_name=sheet_name2, index=False)
    df3.to_excel(writer, sheet_name=sheet_name3, index=False)
    df4.to_excel(writer, sheet_name=sheet_name4, index=False)
    df5.to_excel(writer, sheet_name=sheet_name5, startrow=4, header=False, index=False)
    df6.to_excel(writer, sheet_name=sheet_name6, startrow=4, header=False, index=False)
    df7.to_excel(writer, sheet_name=sheet_name7, startrow=4, header=False, index=False)
    df8.to_excel(writer, sheet_name=sheet_name8, startrow=4, header=False, index=False)
    df9.to_excel(writer, sheet_name=sheet_name9, startrow=4, header=False, index=False)
    df10.to_excel(writer, sheet_name=sheet_name10, startrow=4, header=False, index=False)
    df11.to_excel(writer, sheet_name=sheet_name11, startrow=4, header=False, index=False)
    df12.to_excel(writer, sheet_name=sheet_name12, startrow=4, header=False, index=False)
    df13.to_excel(writer, sheet_name=sheet_name13, startrow=4, header=False, index=False)
    df14.to_excel(writer, sheet_name=sheet_name14, startrow=4, header=False, index=False)
    df15.to_excel(writer, sheet_name=sheet_name15, startrow=4, header=False, index=False)
    df16.to_excel(writer, sheet_name=sheet_name16, startrow=4, header=False, index=False)
    
    # Aplicar a formatação às sheets com os DataFrames
    formatar_sheet(writer.sheets[sheet_name_trim], sheet_name_trim)
    formatar_sheet(writer.sheets[sheet_name2], sheet_name2)
    formatar_sheet(writer.sheets[sheet_name3], sheet_name3)
    formatar_sheet(writer.sheets[sheet_name4], sheet_name4)
    formatar_sheet(writer.sheets[sheet_name5], sheet_name5)
    
    formatar_sheet(writer.sheets[sheet_name6], sheet_name6)
    formatar_sheet(writer.sheets[sheet_name7], sheet_name7)
    formatar_sheet(writer.sheets[sheet_name8], sheet_name8)
    formatar_sheet(writer.sheets[sheet_name9], sheet_name9)
    formatar_sheet(writer.sheets[sheet_name10], sheet_name10)
    formatar_sheet(writer.sheets[sheet_name11], sheet_name11)
    formatar_sheet(writer.sheets[sheet_name12], sheet_name12)
    formatar_sheet(writer.sheets[sheet_name13], sheet_name13)
    formatar_sheet(writer.sheets[sheet_name14], sheet_name14)
    formatar_sheet(writer.sheets[sheet_name15], sheet_name15)
    formatar_sheet(writer.sheets[sheet_name16], sheet_name16)
    
    